In [ ]:
import sys
print(sys.executable)

In [ ]:
import sys
import os
import json

# Get the parent directory of the current notebook
base_folder = os.path.abspath("..")
print(f"Your base folder is: {base_folder}")
sys.path.append(base_folder)

In [3]:
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

import multiprocessing
from tqdm import tqdm
import numpy as np

from data import get_wikitext_data, get_fineweb_data, save_data, load_data, clean_textdata
from tokenization import get_tiktoken_tokenizer

In [ ]:
batch_size = 64  # Kept the same; could be adjusted based on hardware
block_size = 1024  # GPT-2 uses a context length of 1024 tokens
max_iters = 50000  # More iterations needed for larger models
eval_interval = 1000  # Increase since more iterations are done
learning_rate = 5e-5  # GPT-2 uses a lower learning rate
eval_iters = 500  # More frequent evaluation for stability

n_embd = 768  # GPT-2 uses 768 for the small version, 1024 for medium, 1280 for large, 1600 for XL
n_head = 12  # GPT-2 uses 12 attention heads
n_layer = 12  # GPT-2 has 12 transformer blocks in the small version
dropout = 0.1  # GPT-2 uses 0.1 dropout for better generalization
print(device)

In [ ]:
DATA_PATH = f"{base_folder}/data/tiktoken_tokenized_wikitext"
num_cores = multiprocessing.cpu_count()

tokenizer = get_tiktoken_tokenizer()
dataset = get_wikitext_data()
# save_data(dataset, DATA_PATH)

print(tokenizer)
print(dataset)

In [ ]:
def clean_batch(examples):
    cleaned_texts = [clean_textdata(text) for text in examples["text"]]
    return {"text": cleaned_texts}

# Clean the entire dataset
cleaned_dataset = dataset.map(
    clean_batch,
    batched=True,
    batch_size=1000,
    num_proc=num_cores,
    desc="Cleaning text"
)

dataset["train"][0]["text"][:100]
cleaned_dataset["train"][0]["text"][:100]

print(cleaned_dataset)


In [ ]:
print(cleaned_dataset["test"]["text"])

In [ ]:
def tokenize_batch(examples, tokenizer, max_length=256):
    tokens = [tokenizer.encode(text) for text in examples["text"]]
    padded_tokens = []
    for seq in tokens:
        if len(seq) > max_length:
            padded_tokens.append(seq[:max_length])
        else:
            padded_tokens.append(seq + [0] * (max_length - len(seq)))

    return {"tokens": padded_tokens}

tokenized_dataset = cleaned_dataset.map(
    tokenize_batch,
    fn_kwargs={"tokenizer": tokenizer},
    batched=True,
    batch_size=1000,
    num_proc=num_cores,
    remove_columns=["text"],
    desc="Tokenizing"
)

tokenized_dataset_text = tokenized_dataset.filter(lambda x: len(x["tokens"]) > 0)

print(tokenized_dataset_text["train"][1])


In [ ]:
print(tokenized_dataset_text)

# data = torch.stack([torch.tensor(tokens, dtype=torch.long) for tokens in tqdm(tokenized_dataset_text["train"]["tokens"], desc="Loading data")])

tokens_array = np.array(tokenized_dataset_text["train"]["tokens"], dtype=np.int64)
data = torch.from_numpy(tokens_array)

print(data.shape, data.dtype)
print(data[:100])